# SHOPRun and SHOPCase

1. Explore SHOPRun
2. Create SHOPCase
3. Trigger SHOPRun from SHOPCase
4. Edit Case File
5. Edit Case Data in Python

## Preparation

Instantiating `PowerOpsClient`...

In [1]:
# You can control which setting files are loaded through the environmental variable below.
# In this case, the setting files are located two levels above, in the root of the repository.
import os
from cognite.powerops import PowerOpsClient

os.environ["SETTINGS_FILES"] = "../../settings.toml;../../.secrets.toml"

powerops = PowerOpsClient.from_settings()

## Explore SHOPRun

`SHOPRun` represents a single attempt at running SHOP.

In [2]:
# Retrieve an existing shop run by event external_id.
shop_run_0 = powerops.shop.retrieve("POWEROPS_SHOP_RUN_2023-10-26T09:56:27.964670Z_efc805")
shop_run_0.check_status()

<SHOPRunStatus.SUCCESS: 'success'>

In [3]:
print(shop_run_0)

{
    "external_id": "POWEROPS_SHOP_RUN_2023-10-26T09:56:27.964670Z_efc805",
    "watercourse": "Fornebu",
    "start": "2022-09-20 22:00:00+00:00",
    "end": "2022-10-02 22:00:00+00:00",
    "case_file_external_id": "cog_shop_preprocessor/cogShop/Case_2023-10-26T09:56:27.964670Z_efc805/44f8e8a9-e7ad-4bff-b544-99d8e7637a9c",
    "shop_files_external_ids": [
        {
            "external_id": "SHOP_Fornebu_water_value_cut_file_reservoir_mapping",
            "file_type": "ascii"
        },
        {
            "external_id": "SHOP_Fornebu_water_value_cut_file",
            "file_type": "ascii"
        }
    ],
    "shop_version": "14.4.3.0"
}


In [4]:
# Small helper to display first 10 lines of a long string (e.g. content of a file).
from IPython.display import display, HTML

def preview_content(content):
    N_LINES = 10
    lines = content.splitlines()
    trimmed_content = "\n".join(lines[:N_LINES])
    if len(lines) > N_LINES:
        trimmed_content += "\n..."
    display(HTML(f"<pre style='font-size: 75%; border: 1px solid var(--jp-content-font-color1); padding: 10px; margin: 10px 0 0 10px;'>{trimmed_content.strip()}</pre>"))

# Same, just for a file.
def preview_file(file_path):
    with open(file_path) as file:
        preview_content(file.read())

In [5]:
# Show first 10 lines of each additional SHOP file in the case.
N_LINES = 10
for shop_file_content in shop_run_0.get_shop_files():
    preview_content(shop_file_content)

# also available:
# shop_run_0.get_case_file()  # content of the case file
# list(shop_run_0.get_log_files())  # content of log files, generated by SHOP

## Create SHOPCase

`SHOPCase` is an editable representation of a SHOP case. It is used to prepare data for a SHOP run.

We run SHOP by "triggering" `SHOPCase`. This creates a new `SHOPRun` instance.

`SHOPCase` can be created from an existing `SHOPRun` instance (using `shop_run.as_case()`) or by directly instantiating the class. The former is probably useful more often.

We will create a new SHOPCase from the existing SHOPRun, then run this new case.

In [6]:
# Create new SHOPCase from a previous SHOPRun.
case_1 = shop_run_0.to_case()
case_1

In [7]:
# Run SHOP by triggering the new case.
shop_run_1 = powerops.shop.trigger_case(case_1)
shop_run_1.check_status()

<SHOPRunStatus.IN_PROGRESS: 'in_progress'>

In [8]:
# A small helper function to wait for a shop run to finish.
from time import sleep


def wait(shop_run):
    while shop_run.check_status() == "in_progress":
        sleep(1)
        print(".", end="")
    print("DONE")


wait(shop_run_1)
shop_run_1.check_status()

.............................DONE


<SHOPRunStatus.SUCCESS: 'success'>

## Edit Case File

We will save the case YAML file to local filesystem, then make some changes to it and use the changed file to run SHOP again. 

The case file is here edited using Python, but in real use it is expected that the file be edited using external tools.

In [9]:
# Make a new SHOPCase and save the case file to local filesystem.
case = shop_run_1.to_case()
case.save_yaml("case_file.yaml")

In [10]:
preview_file("case_file.yaml")

In [11]:
# Deliberately break the YAML structure of the file.
# Here we delete first 15 lines of the file for demonstration purposes.
with open("case_file.yaml") as case_file:
    lines = case_file.readlines()
with open("case_file.yaml", "w") as case_file:
    case_file.writelines(lines[15:])
preview_file("case_file.yaml")

In [12]:
# This is now an invalid YAML file, so we cannot use it with SHOPCase.
try:
    case.load_case_file("case_file.yaml")
except Exception as exc:
    print(repr(exc))

ValueError('Could not parse case data')


In [13]:
# Add "commands:" line to the beginning of the file.
# This will fix the YAML structure, but the case itself will remain broken.
broken_case = open("case_file.yaml").read()
with open("case_file.yaml", "w") as case_file:
    case_file.write("commands:\n")
    case_file.write(broken_case)
preview_file("case_file.yaml")

In [14]:
# Now we have a technically valid YAML, we can use it to run SHOP.
case.load_case_file("case_file.yaml")
shop_run_2 = powerops.shop.trigger_case(case)
shop_run_2.check_status()

<SHOPRunStatus.IN_PROGRESS: 'in_progress'>

In [15]:
# We expect SHOP run to run and fail, though.
wait(shop_run_2)
shop_run_2.check_status()

................DONE


<SHOPRunStatus.FAILED: 'failed'>

In [16]:
# More information about the failure can be obtained from the SHOP run object.
from pprint import pprint
pprint(shop_run_2.get_failure_info())

{'error': 'Traceback (most recent call last):\n'
          '  File "/workspace/cogshop/status_events.py", line 84, in wrapper\n'
          '    res = func(client, event_external_id, *args, **kwargs)\n'
          '  File "/workspace/cogshop/main.py", line 70, in _main\n'
          '    raise CogShopError(cog_shop.solver_status)\n'
          'cogshop.exceptions.CogShopError: Integer infeasible\n',
 'failures': None}


In [17]:
shop_run_2

,Value
external_id,SHOP_RUN_2023-10-28T10:59:32.508289Z_cf0844
watercourse,Fornebu
start,2023-10-28 10:59:32.508289+00:00
end,None
case_file_external_id,cog_shop_manual_2023-10-28T10:59:32.508289Z_2a...
shop_files_external_ids,[{'external_id': 'SHOP_Fornebu_water_value_cut...
shop_version,CogShop2-20230710T163856Z


## Edit Data in Python

Besides editing the case file and using `case.load_case_file()`, we can also interact with case data directly, using `case.data` dict.

We will fix the case data by adding the missing commands back.

In [18]:
# Add missing commmands back, direclty into case data.
case.data["commands"] = [
    "set time_delay_unit MINUTE",
    "set ramping /on",
    "set stop_cost_from_start_cost /on",
    "set bypass_loss /on",
    "set mipgap 0.001000",
    "set timelimit 600.000",
    "set reserve_ramping_cost 1",
    "set fcr_n_equality /on",
    "set reserve_slack_cost 1",
    "set reserve_min_capacity 0.02",
    "set dyn_seg /on",
    "set dyn_juncloss /on",
    "penalty flag /on /plant /schedule",
    "start sim 3",
    "set code /inc",
    "start sim 4",
]

In [19]:
# This time we expect the SHOP run to succeed.
shop_run_3 = powerops.shop.trigger_case(case)
wait(shop_run_3)
shop_run_3.check_status()

............................DONE


<SHOPRunStatus.SUCCESS: 'success'>

## Cleanup

In [20]:
# Remove the local case file.
import os

os.unlink("case_file.yaml")

End of notebook.